FakeBrCorpus:
- Dataset: https://github.com/roneysco/Fake.br-Corpus
- Docs: https://sites.icmc.usp.br/taspardo/PROPOR2018-MonteiroEtAl.pdf

TwitterBR:
- Dataset: https://github.com/phfaustini/BRACIS2019_FAKENEWS
- Docs: https://ieeexplore.ieee.org/document/8923888/footnotes#footnotes

fake_real_news_dataset:
- Dataset: https://github.com/GeorgeMcIntire/fake_real_news_dataset
- Docs: https://ieeexplore.ieee.org/abstract/document/8257971/footnotes#footnotes

Fakenewsdata1
- Dataset: https://github.com/rpitrust/fakenewsdata1
- Docs: https://arxiv.org/pdf/1703.09398.pdf

News-credibility
- Dataset: https://github.com/mhardalov/news-credibility
- Docs: https://arxiv.org/pdf/1911.08125.pdf


In [107]:
import os, re
import gensim, nltk
import pandas as pd
import numpy as np
from nltk.test.gensim_fixt import setup_module
from gensim.models import word2vec

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [2]:
setup_module()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/knguyen02311/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
PROJECT_DIR = '/media/knguyen02311/Data Disk/DataScience/data_mining/is252/TermProject/'
STOP_WORDS = nltk.corpus.stopwords.words()

In [4]:
fake_br_corpus_df = pd.read_csv(os.path.join(PROJECT_DIR, 'dataset/Fake.br-Corpus/preprocessed/pre-processed.csv'))

In [5]:
fake_br_corpus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              7200 non-null   int64 
 1   label              7200 non-null   object
 2   preprocessed_news  7200 non-null   object
dtypes: int64(1), object(2)
memory usage: 168.9+ KB


In [6]:
fake_br_corpus_df.loc[0, :]["preprocessed_news"]

'katia abreu diz vai colocar expulsao moldura nao reclamar senadora katia abreu disse expulsao pmdb resultado acao cupula atual legenda segundo oportunista amanha vou botar moldura dourada expulsao porque maos onde veio atestado boa conduta curriculo pessoas expulsaram nao servem pais servem pais beneficios proprios disse katia abreu ue expulsao algo tao bom curriculo tanta choradeira katia sabemos motivo provavelmente katia nao valor pt partido ja deveria absorvido parece pt gostava katia somente ficasse entrincheirada dentro pmdb rebaixar demais resta katia ficar chorando pitangas todos cantos tempo ate momento pt nao cadastrou katia abreu fileiras situacao patetica agricultura dilma'

In [7]:
fake_br_corpus_df.sample(10)

,index,label,preprocessed_news
6353,6353,true,odebrecht doou r milhao maguito daniel vilela ...
3033,3033,fake,guerra nuclear pode comecar qualquer momento t...
2020,2020,fake,expressao lula diz tudo acabou sonho bandidos ...
1022,1022,fake,anatel afirma prioridade universalizacao banda...
2677,2677,fake,integrantes mst confessam governo aluga varios...
6674,6674,true,buraco negro informacao seguranca publica bras...
3712,3712,true,anthony garotinho ouvido vez sobre suposta agr...
4696,4696,true,primeira turma stf discute nesta terca pedido ...
5798,5798,true,julio andrade explica atuar via mao dupla inte...
682,682,fake,pastor igreja eike irrita dispara seguido cris...


In [96]:
def one_hot_label(label):
   return 1 if label == 'true' else 0

def build_corpus(data):
   "Creates a list of lists containing words from each sentence"
   corpus = []
   for col in ['preprocessed_news']:
      for sentence in data[col].items():
         word_list = sentence[1].split(" ")
         corpus.append(word_list)
   return corpus


def vectorize(splited_sentence, w2v_model):
   # words_vecs = []
   # for word in splited_sentence:
   #    if word in w2v_model.wv:
   #       words_vecs.append(w2v_model.wv[word])
   #    else:
   #       words_vecs.append(np.zeros((100)))
   # return words_vecs
   # # return words_vecs.mean(axis=0)
   
   words_vecs = [w2v_model.wv[word] for word in splited_sentence if word in w2v_model.wv]
   if len(words_vecs) == 0:
      return np.zeros(100)
   words_vecs = np.array(words_vecs)
   return words_vecs.mean(axis=0)



In [91]:
corpus = build_corpus(fake_br_corpus_df)
len(corpus)

7200

In [10]:
word2vec_model = word2vec.Word2Vec(corpus, vector_size=100, window=10, min_count=2, workers=4)

In [38]:
word2vec_model.wv['curriculo']

array([-0.00592661,  0.11762395, -0.047673  ,  0.2911397 , -0.09126366,
       -0.1467089 ,  0.12433521,  0.51715237, -0.37797466, -0.05832652,
        0.18548086, -0.20148583,  0.1525664 ,  0.07050457, -0.20804682,
       -0.29897416,  0.01424311, -0.01313357, -0.03835886, -0.44936225,
        0.19281858,  0.0565613 ,  0.1801666 ,  0.0044343 ,  0.00378894,
        0.16297264, -0.22955807,  0.11046262, -0.35011488,  0.09588832,
        0.07435477,  0.11931235, -0.233192  , -0.09000166,  0.05756364,
       -0.01630505,  0.19297482, -0.21479686, -0.23306113, -0.5257807 ,
       -0.14618424, -0.47525924,  0.04754307,  0.24647847,  0.18877034,
       -0.15178062, -0.50294495, -0.03916204, -0.11257908, -0.06662774,
       -0.08864307, -0.15645152, -0.1386384 , -0.05662479, -0.19408882,
        0.24928996,  0.27915874,  0.17389366, -0.2708606 , -0.10224747,
        0.05099581,  0.12525883, -0.11591211,  0.00185504,  0.14562128,
        0.36829522,  0.0059851 ,  0.40124655, -0.00548349,  0.06

In [12]:
fake_br_corpus_df['label'].apply(one_hot_label).to_numpy().reshape(-1, 1)

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [104]:
x = np.array([vectorize(sentence, word2vec_model) for sentence in corpus])
y = fake_br_corpus_df['label'].apply(one_hot_label).to_numpy().ravel()

x.shape, y.shape

((7200, 100), (7200,))

In [108]:
def cross_validation(model, _X, _y, folds=5):
      _scoring = ['accuracy', 'precision', 'recall', 'f1']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=folds,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Training Accuracy scores": results['train_accuracy'],
              "Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Training Precision scores": results['train_precision'],
              "Mean Training Precision": results['train_precision'].mean(),
              "Training Recall scores": results['train_recall'],
              "Mean Training Recall": results['train_recall'].mean(),
              "Training F1 scores": results['train_f1'],
              "Mean Training F1 Score": results['train_f1'].mean(),
              "Validation Accuracy scores": results['test_accuracy'],
              "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
              "Validation Precision scores": results['test_precision'],
              "Mean Validation Precision": results['test_precision'].mean(),
              "Validation Recall scores": results['test_recall'],
              "Mean Validation Recall": results['test_recall'].mean(),
              "Validation F1 scores": results['test_f1'],
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

In [118]:
model_naivebayes = BernoulliNB()
model_knn = KNeighborsClassifier(n_neighbors=7)
model_svm = SVC(kernel='linear', gamma='auto')
model_rf = RandomForestClassifier(n_estimators=951)

print('NB:', cross_validation(model_naivebayes, x, y)['Mean Validation Accuracy'])
print('KNN:', cross_validation(model_knn, x, y)['Mean Validation Accuracy'])
print('SVM:', cross_validation(model_svm, x, y)['Mean Validation Accuracy'])
print('RF:', cross_validation(model_rf, x, y)['Mean Validation Accuracy'])

NB: 68.33333333333333
KNN: 67.54166666666667
SVM: 79.94444444444446
RF: 84.625
